# DEPURACIÓN Y CREACIÓN DE NUEVA BASE DE DATOS

## 1. DEPURACIÓN

In [ ]:
# CARGA DE LIBRERÍAS

import pandas as pd

In [ ]:
# CARGA DE LA BASE DE DATOS

df = pd.read_csv('Registre_d_entitats__serveis_i_establiments_socials__serveis_socials_b_sics_i_especialitzats__20240515.csv', sep=',')

print (df)

In [ ]:
# Obtención de los diferentes valores de la columna 'Tipologia'
valores_tipologia = df['Tipologia'].unique()

# Imprimir los diferentes valores
print("Diferentes valores de la columna 'Tipologia':")
print(valores_tipologia)


In [ ]:
# Lista de valores a conservar en la columna 'Tipologia'

valores_a_conservar = [
    'Servei de centre residencial temporal o permanent per a persones amb discapacitat intel·lectual',
    'Servei de llar residència temporal o permanent per a persones amb discapacitat intel·lectual',
    'Servei de llar amb suport per a persones amb discapacitat intel·lectual'
]

# Filtrado del dataframe para conservar sólo las filas con los valores especificados en la lista
df = df[df['Tipologia'].isin(valores_a_conservar)]

In [ ]:
# Eliminación de la columna del índice si no se ha nombrado correctamente
if 'Unnamed: 0' in df.columns:
    del df.index.name

# Lista de columnas a eliminar
columnas_a_eliminar = ['Adreça', 'Telèfon', 'Districte', 'CP', 'NIF']

# Eliminación de las columnas especificadas
df = df.drop(columns=columnas_a_eliminar)

In [ ]:
# Eliminación del espacio en blanco adicional del nombre de la columna 'Inscripció'
df.rename(columns=lambda x: x.strip(), inplace=True)

# Imprimir los nombres de las columnas actualizados
print(df.columns)



In [ ]:
# Cambiar el nombre de la columna 'Inscripció' a 'Any'
df.rename(columns={"Inscripció": "Any"}, inplace=True)

print("\nDataFrame modificado:")
print(df)
# Imprimir los nombres de las columnas
print(df.columns)


In [ ]:
# Conversión de la columna 'Inscripció' a formato de fecha
df['Any'] = pd.to_datetime(df['Any'])

# Extracción sólo del año y asignación a la columna 'Inscripció'
df['Any'] = df['Any'].dt.year

# Imprimir el dataframe actualizado
print(df)



In [ ]:
# Eliminación del índice predeterminado
df.reset_index(drop=True, inplace=True)

print (df)

In [ ]:
# Definir el diccionario de mapeo para los cambios de nombres
nuevos_nombres = {
    'Servei de centre residencial temporal o permanent per a persones amb discapacitat intel·lectual': 'Residència',
    'Servei de llar amb suport per a persones amb discapacitat intel·lectual': 'Llar amb suport',
    'Servei de llar residència temporal o permanent per a persones amb discapacitat intel·lectual': 'Llar residència'
}

# Aplicación de los cambios de nombres utilizando el método replace()
df['Tipologia'] = df['Tipologia'].replace(nuevos_nombres)


In [ ]:
# Creación del archivo 'habitatge1.csv'
df.to_csv('habitatge1.csv', index=False)

## 2.CREACIÓN DE BASE DE DATOS CON DATOS ACUMULADOS

In [ ]:
df2=df

In [ ]:
# Obtención de todas las combinaciones posibles de año, comarca y tipología
all_years = df2['Any'].unique()
all_comarcas = df2['Comarca'].unique()
all_tipologias = df2['Tipologia'].unique()

all_combinations = []
for year in all_years:
    for comarca in all_comarcas:
        for tipologia in all_tipologias:
            all_combinations.append((year, comarca, tipologia))

# Cálculo de las combinaciones faltantes
existing_combinations = set(zip(df2['Any'], df2['Comarca'], df2['Tipologia']))
missing_combinations = [combo for combo in all_combinations if combo not in existing_combinations]

# Creación de un DataFrame con las combinaciones faltantes
missing_data = pd.DataFrame(missing_combinations, columns=['Any', 'Comarca', 'Tipologia'])
missing_data['Habitatges'] = 0
missing_data['Capacitat'] = 0

# Concatenación del DataFrame nuevo con las combinaciones faltantes
final_df = pd.concat([df2, missing_data])

# Agrupación por año, comarca y tipología y cálculo de la suma de la capacidad y el recuento de registros
final_df = final_df.groupby(['Any', 'Comarca', 'Tipologia']).agg({'Capacitat': 'sum', 'Registre': 'count'}).reset_index()

# Renombrado de la columna 'Registre' como 'Habitatges'
final_df.rename(columns={'Registre': 'Habitatges'}, inplace=True)

# Ordenación del DataFrame final
final_df = final_df.sort_values(by=['Any', 'Comarca', 'Tipologia']).reset_index(drop=True)

# Reordenado de las columnas
final_df = final_df[['Any', 'Comarca', 'Tipologia', 'Habitatges', 'Capacitat']]

print (final_df)

In [ ]:
# Obtención de los diferentes valores de la columna 'Qualificació'
valores_qualificacio = df2['Qualificació'].unique()

# Cálculo de las ocurrencias de cada variable en 'Qualificació' en función de 'Any', 'Comarca' y 'Tipologia'
nuevas_columnas = df2.groupby(['Any', 'Comarca', 'Tipologia'])['Qualificació'].value_counts().unstack(fill_value=0)

# Renombrado de las columnas resultantes con los valores de 'Qualificació'
nuevas_columnas.columns = [f'{valor}_count' for valor in valores_qualificacio]

# Reseteo del índice para convertir los niveles del índice en columnas
nuevas_columnas.reset_index(inplace=True)

# Unión del DataFrame original con las nuevas columnas
final_df = pd.merge(final_df, nuevas_columnas, on=['Any', 'Comarca', 'Tipologia'], how='left')

print (final_df)

In [ ]:
# Lista de comarcas faltantes
comarcas_faltantes = ["Alta Ribagorça", "Lluçanès", "Pallars Sobirà", "Segarra", "Terra Alta", "Val d'Aran"]

# Creación de un DataFrame con las comarcas faltantes
df_comarcas_faltantes = pd.DataFrame(columns=['Any', 'Comarca', 'Tipologia', 'Habitatges', 'Capacitat'])

# Llenado del DataFrame con las comarcas faltantes
for comarca in comarcas_faltantes:
    for year in df2['Any'].unique():
        for tipologia in df2['Tipologia'].unique():
            df_comarcas_faltantes = pd.concat([df_comarcas_faltantes, pd.DataFrame({'Any': [year], 'Comarca': [comarca], 'Tipologia': [tipologia], 'Habitatges': [0], 'Capacitat': [0]})], ignore_index=True)

# Fusión de df_final con df_comarcas_faltantes
df_final = pd.concat([final_df, df_comarcas_faltantes])

# Reemplazo de NaN por 0 en las columnas 'Habitatges' y 'Capacitat'
df_final['Habitatges'] = df_final['Habitatges'].fillna(0)
df_final['Capacitat'] = df_final['Capacitat'].fillna(0)

# Conversión de las columnas 'Habitatges' y 'Capacitat' a tipo numérico
df_final['Habitatges'] = pd.to_numeric(df_final['Habitatges'])
df_final['Capacitat'] = pd.to_numeric(df_final['Capacitat'])

# Ordenación del DataFrame final
df_final = df_final.sort_values(by=['Any', 'Comarca', 'Tipologia']).reset_index(drop=True)

print (df_final)


In [ ]:
# Cálculo de la suma acumulada para 'Habitatges' y 'Capacitat' por 'Comarca' y 'Tipologia'
df_final['Habitatge acumulat'] = df_final.groupby(['Comarca', 'Tipologia'])['Habitatges'].cumsum()
df_final['Capacitat acumulada'] = df_final.groupby(['Comarca', 'Tipologia'])['Capacitat'].cumsum()


print(df_final)


In [ ]:
# Creación del archivo 'Habitatge2.csv'
df_final.to_csv('Habitatge2.csv', index=False)